# **Задание по теме 3.**
***На основе представленных в файле credit.csv данных решить задачу скоринга. Возможные варианты инструментов: нейросеть, дерево решений, лес деревьев решений, логистическая регрессия. Построить не менее 2 сетей с разной архитектурой. Сделать обоснованный выбор в пользу одного из инструментов на основе метрик ошибки.***

## ***1. Загрузка данных***

In [3]:
import pandas as pd

df = pd.read_csv('credit.csv', delimiter=';', encoding='cp1251')

## ***2. Предварительный анализ данных***

### ***2.1. Обзор структуры данных***

In [4]:
df.head()

,client_id,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30.59Delinquencies,Num60.89Delinquencies,Income,BalanceToCreditLimit,Delinquent90
0,33,"0,229227","34,2",5,0,3,0,0,"4365,787145","0,614501",0
1,34,"1,088303","31,3",10,2,1,0,0,"3042,93144","0,33446",1
2,48,0,"40,3",12,0,0,0,0,"5236,071398","4,80E-05",0
3,51,"0,085763","62,4",5,0,0,0,0,"7715,104911","0,367092",0
4,53,"0,340004","31,1",14,1,0,0,0,"10987,84611","0,394608",0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9803 entries, 0 to 9802
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   client_id              9803 non-null   int64 
 1   DIR                    9803 non-null   object
 2   Age                    9803 non-null   object
 3   NumLoans               9803 non-null   int64 
 4   NumRealEstateLoans     9803 non-null   int64 
 5   NumDependents          9803 non-null   int64 
 6   Num30.59Delinquencies  9803 non-null   int64 
 7   Num60.89Delinquencies  9803 non-null   int64 
 8   Income                 9803 non-null   object
 9   BalanceToCreditLimit   9803 non-null   object
 10  Delinquent90           9803 non-null   int64 
dtypes: int64(7), object(4)
memory usage: 842.6+ KB


### ***2.2. Проверка на отсутствующие значения***

In [6]:
df.isnull().sum()

client_id                0
DIR                      0
Age                      0
NumLoans                 0
NumRealEstateLoans       0
NumDependents            0
Num30.59Delinquencies    0
Num60.89Delinquencies    0
Income                   0
BalanceToCreditLimit     0
Delinquent90             0
dtype: int64

## ***3. Подготовка данных***

In [10]:
for column in df.select_dtypes(include='object').columns:
    df[column] = df[column].str.replace(',', '.').astype(float)

df.head()

,client_id,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30.59Delinquencies,Num60.89Delinquencies,Income,BalanceToCreditLimit,Delinquent90
0,33,0.229227,34.2,5,0,3,0,0,4365.787145,0.614501,0
1,34,1.088303,31.3,10,2,1,0,0,3042.931440,0.334460,1
2,48,0.000000,40.3,12,0,0,0,0,5236.071398,0.000048,0
3,51,0.085763,62.4,5,0,0,0,0,7715.104911,0.367092,0
4,53,0.340004,31.1,14,1,0,0,0,10987.846110,0.394608,0


In [11]:
from sklearn.model_selection import train_test_split

x = df.drop('Delinquent90', axis=1)
y = df['Delinquent90']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

## ***4. Построение и обучение моделей***

In [13]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

### ***4.1. Нейросеть с первой архитектурой***

In [14]:
model_nn1 = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(x_train_scaled.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model_nn1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_nn1.fit(x_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(x_test_scaled, y_test))


Metal device set to: Apple M1 Max
Epoch 1/10


2023-05-16 20:51:08.766067: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


246/246 [==============================] - 5s 11ms/step - loss: 0.2507 - accuracy: 0.9267 - val_loss: 0.1939 - val_accuracy: 0.9363
Epoch 2/10
246/246 [==============================] - 2s 7ms/step - loss: 0.1919 - accuracy: 0.9390 - val_loss: 0.1908 - val_accuracy: 0.9393
Epoch 3/10
246/246 [==============================] - 2s 8ms/step - loss: 0.1877 - accuracy: 0.9416 - val_loss: 0.1865 - val_accuracy: 0.9424
Epoch 4/10
246/246 [==============================] - 2s 8ms/step - loss: 0.1840 - accuracy: 0.9422 - val_loss: 0.1858 - val_accuracy: 0.9388
Epoch 5/10
246/246 [==============================] - 2s 8ms/step - loss: 0.1812 - accuracy: 0.9422 - val_loss: 0.1887 - val_accuracy: 0.9368
Epoch 6/10
246/246 [==============================] - 2s 8ms/step - loss: 0.1804 - accuracy: 0.9434 - val_loss: 0.1862 - val_accuracy: 0.9414
Epoch 7/10
246/246 [==============================] - 2s 7ms/step - loss: 0.1785 - accuracy: 0.9430 - val_loss: 0.1855 - val_accuracy: 0.9414
Epoch 8/10
246/2

### ***4.2. Дерево решений***

In [15]:
model_tree = DecisionTreeClassifier()
model_tree.fit(x_train_scaled, y_train)

DecisionTreeClassifier()

### ***4.3. Лес деревьев решений***

In [16]:
model_forest = RandomForestClassifier()
model_forest.fit(x_train_scaled, y_train)

RandomForestClassifier()

### ***4.4. Логистическая регрессия***

In [17]:
model_lr = LogisticRegression()
model_lr.fit(x_train_scaled, y_train)

LogisticRegression()

### ***4.5. Нейронная сеть со второй архитектурой***

In [18]:
model_nn2 = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(x_train_scaled.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model_nn2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_nn2.fit(x_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(x_test_scaled, y_test))

Epoch 1/10
246/246 [==============================] - 3s 10ms/step - loss: 0.2244 - accuracy: 0.9343 - val_loss: 0.1915 - val_accuracy: 0.9393
Epoch 2/10
246/246 [==============================] - 2s 8ms/step - loss: 0.1891 - accuracy: 0.9385 - val_loss: 0.1866 - val_accuracy: 0.9398
Epoch 3/10
246/246 [==============================] - 2s 8ms/step - loss: 0.1822 - accuracy: 0.9422 - val_loss: 0.1844 - val_accuracy: 0.9414
Epoch 4/10
246/246 [==============================] - 2s 8ms/step - loss: 0.1800 - accuracy: 0.9408 - val_loss: 0.1859 - val_accuracy: 0.9414
Epoch 5/10
246/246 [==============================] - 2s 8ms/step - loss: 0.1788 - accuracy: 0.9436 - val_loss: 0.1877 - val_accuracy: 0.9398
Epoch 6/10
246/246 [==============================] - 2s 9ms/step - loss: 0.1760 - accuracy: 0.9450 - val_loss: 0.1882 - val_accuracy: 0.9393
Epoch 7/10
246/246 [==============================] - 2s 8ms/step - loss: 0.1732 - accuracy: 0.9444 - val_loss: 0.1880 - val_accuracy: 0.9434
Epoch

## ***5. Оценка производительности***

In [19]:
from sklearn.metrics import classification_report

### ***5.1. Оценка первой нейронной сети***

In [32]:
y_pred_nn1 = model_nn1.predict(x_test_scaled)
y_pred_nn1 = (y_pred_nn1 > 0.5).astype(int)
classification_report_nn1 = classification_report(y_test, y_pred_nn1)

print("Метрики ошибки для первой нейросети (Model_nn1):")
print(classification_report_nn1)

62/62 [==============================] - 0s 2ms/step
Метрики ошибки для первой нейросети (Model_nn1):
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      1829
           1       0.65      0.27      0.38       132

    accuracy                           0.94      1961
   macro avg       0.80      0.63      0.67      1961
weighted avg       0.93      0.94      0.93      1961



### ***5.2. Оценка второй нейронной сети***

In [22]:
y_pred_nn2 = model_nn2.predict(x_test_scaled)
y_pred_nn2 = (y_pred_nn2 > 0.5).astype(int)
classification_report_nn2 = classification_report(y_test, y_pred_nn2)

print("Метрики ошибки для второй нейросети (Model_nn2):")
print(classification_report_nn2)

62/62 [==============================] - 0s 2ms/step
Метрики ошибки для второй нейросети (Model_nn2):
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      1829
           1       0.72      0.21      0.33       132

    accuracy                           0.94      1961
   macro avg       0.83      0.60      0.65      1961
weighted avg       0.93      0.94      0.93      1961



### ***5.3. Оценка дерева решений***

In [23]:
y_pred_tree = model_tree.predict(x_test_scaled)
classification_report_tree = classification_report(y_test, y_pred_tree)

print("Метрики ошибки для дерева решений:")
print(classification_report_tree)

Метрики ошибки для дерева решений:
              precision    recall  f1-score   support

           0       0.95      0.94      0.94      1829
           1       0.28      0.34      0.31       132

    accuracy                           0.90      1961
   macro avg       0.62      0.64      0.63      1961
weighted avg       0.91      0.90      0.90      1961



### ***5.4. Оценка леса деревьев решений***

In [24]:
y_pred_forest = model_forest.predict(x_test_scaled)
classification_report_forest = classification_report(y_test, y_pred_forest)

print("Метрики ошибки для леса деревьев решений:")
print(classification_report_forest)

Метрики ошибки для леса деревьев решений:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      1829
           1       0.61      0.25      0.35       132

    accuracy                           0.94      1961
   macro avg       0.78      0.62      0.66      1961
weighted avg       0.93      0.94      0.93      1961



### ***5.5. Оценка логистической регрессии***

In [25]:
y_pred_lr = model_lr.predict(x_test_scaled)
classification_report_lr = classification_report(y_test, y_pred_lr)

print("Метрики ошибки для логистической регрессии:")
print(classification_report_lr)

Метрики ошибки для логистической регрессии:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      1829
           1       0.71      0.26      0.38       132

    accuracy                           0.94      1961
   macro avg       0.83      0.62      0.67      1961
weighted avg       0.93      0.94      0.93      1961



## ***6. Сравнение результатов***

In [43]:
target_names = ['0', '1']

report_nn1 = classification_report(y_test, y_pred_nn1, output_dict=True)
report_nn2 = classification_report(y_test, y_pred_nn2, output_dict=True)
report_tree = classification_report(y_test, y_pred_tree, target_names=target_names, output_dict=True)
report_forest = classification_report(y_test, y_pred_forest, target_names=target_names, output_dict=True)
report_lr = classification_report(y_test, y_pred_lr, target_names=target_names, output_dict=True)

data = {'Модель': ['Model_nn1', 'Model_nn2', 'Дерево решений', 'Лес деревьев решений', 'Логистическая регрессия'],
        'Точность': [report_nn1['1']['precision'], report_nn2['1']['precision'], report_tree['1']['precision'], report_forest['1']['precision'], report_lr['1']['precision']],
        'Полнота': [report_nn1['1']['recall'], report_nn2['1']['recall'], report_tree['1']['recall'], report_forest['1']['recall'], report_lr['1']['recall']],
        'F1-мера': [report_nn1['1']['f1-score'], report_nn2['1']['f1-score'], report_tree['1']['f1-score'], report_forest['1']['f1-score'], report_lr['1']['f1-score']]}
df = pd.DataFrame(data)

df

,Модель,Точность,Полнота,F1-мера
0,Model_nn1,0.648148,0.265152,0.376344
1,Model_nn2,0.717949,0.212121,0.327485
2,Дерево решений,0.281250,0.340909,0.308219
3,Лес деревьев решений,0.611111,0.250000,0.354839
4,Логистическая регрессия,0.708333,0.257576,0.377778


## 7. ***Выводы***

* **Model_nn1 (первая нейросеть)** показывает низкую ***точность (0.648148)***, ***полноту (0.265152)*** и ***F1-меру (0.376344)***. Это означает, что модель имеет трудности в правильном определении проблемных клиентов, и доля ложных срабатываний (ложноположительных прогнозов) высока.

* **Model_nn2 (вторая нейросеть)** также демонстрирует низкую ***точность (0.717949)***, ***полноту (0.212121)*** и ***F1-меру (0.327485)***. В сравнении с Model_nn1, она показывает незначительное улучшение в точности, но все еще имеет слабую способность обнаруживать проблемных клиентов.

* **Дерево решений** имеет низкую ***точность (0.281250)***, но относительно высокую ***полноту (0.340909)*** и ***F1-меру (0.308219)***. Оно также страдает от низкой точности в предсказании проблемных клиентов.

* **Лес деревьев** решений показывает лучшие результаты с ***точностью (0.611111)***, ***полнотой (0.250000)*** и ***F1-мерой (0.354839)*** в сравнении с предыдущими моделями, но все еще далеко от идеальных показателей.

* **Логистическая регрессия** демонстрирует наилучшие результаты с ***точностью (0.708333)***, ***полнотой (0.257576)*** и ***F1-мерой (0.377778)***. Она показывает более сбалансированные результаты и лучше справляется с предсказанием проблемных клиентов.

***Логистическая регрессия*** показывает наилучшую производительность среди представленных моделей. Ее высокая точность и F1-мера указывают на хорошую способность модели предсказывать проблемных клиентов, а также учитывать баланс между точностью и полнотой.

Таким образом, на основе метрик ошибки, можно сделать выбор в пользу логистической регрессии как наиболее эффективного инструмента для решения задачи скоринга в данном случае.